<h1>Clinical Trials Data Analysis</h1>

<h2>Introduction</h2>

In this project, I will attempt to find the best hospital(s) to select 200 patients for clinical trial with the following conditions:

1. Aged 50 or above<br>
2. Has a peak flow reading above 400 litres per minute (litres/min)<br>
3. Has not had a flu case or flu shot in the last 6 months<br>
Does not have an egg allergy<br>
•	Does not have a history of respiratory conditions<br>
•	Has a BMI (Body Mass Index) between 18-32 kg/m<sup>2</sup><br>
•	Non-smokers only<br>

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# The excel spreadsheet has been converted to a csv file so that it can be read easily
df = pd.read_csv("../data/hospitals_data.csv")

df.head()

,Unnamed: 0,SITEID,SUBJID,SEX,AGE,RACE,ALLERGY,MEDICAL,SMOKER,LAST_DOSE_FLU,BMI,PEAK_FLOW_1,PEAK_FLOW_2,PEAK_FLOW_3
0,0,Hospital A,GSK0001,M,79,MIXED,NaN,NaN,False,False,26,653,635,651
1,1,Hospital A,GSK0002,M,62,WHITE,NaN,NaN,False,True,35,458,449,438
2,2,Hospital A,GSK0003,M,71,ASIAN,NaN,NaN,True,True,21,370,350,352
3,3,Hospital A,GSK0004,M,73,BLACK,NaN,Heart,False,False,22,414,412,412
4,4,Hospital A,GSK0005,F,66,WHITE,NaN,NaN,False,True,29,632,651,627


<h2>Cleaning the data</h2>

There are a few things I need to do in order to clean the data.

1. Remove the `Unamed` row. This row is probably there due to it being an artefact when being converted from Excel to CSV format, so is unnecassery
2. Remove any duplicate rows
3. Remove any rows with invalid data
4. For the `AGE` row, change any data with obvious typos in it

In [85]:
pre_clean_count = len(df.index)     # Used with `post_clean_count` to show how many rows were dropped

# Removes the unamed column (which is first)
df.drop(columns=[df.columns[0]], axis=1, inplace=True)

# Removes any rows with *invalid* data
# In this case, I am going to deal with typos (such as in AGE column) later
# At first I will remove all items which *have* to be wrong  
valid_data = {
    "SITEID"        : (lambda site_id: pd.notna(site_id)),                                  # The SITEID cannot be N/A
    "SUBJID"        : (lambda subj_id: pd.notna(subj_id)),                                  # The SUBJID cannot be N/A
    "SEX"           : (lambda sex: sex.upper() == "M" or sex.upper() == "F"),               # The SEX can only be "M" or "F"
    "AGE"           : (lambda age: str(age).strip('-').isnumeric()),                        # The AGE must be numeric (allowing negative values to clean)
    "RACE"          : (lambda race: race in ["MIXED", "WHITE", "ASIAN", "BLACK", "OTHER"]), # The RACE can only hvave the values which are in the list
    "ALLERGY"       : (lambda allergy: True),                                               # The ALLERGY can essentially be anything - N/A or a string
    "MEDICAL"       : (lambda medical: True),                                               # The MEDICAL can also be abything N/A or a string
    "SMOKER"        : (lambda smoker: type(smoker) == bool),                                # The smoker can only be a boolean (not N/A)
    "LAST_DOSE_FLU" : (lambda last_dose_flu: type(last_dose_flu) == bool),                  # The last flu shot can also only be a boolean
    "BMI"           : (lambda bmi: str(bmi).isnumeric()),                                   # The BMI can only be numeric as well
    "PEAK_FLOW_1"   : (lambda peak_flow: str(peak_flow).isnumeric()),                       # The PEAK_FLOW columns can also only be numeric
    "PEAK_FLOW_2"   : (lambda peak_flow: str(peak_flow).isnumeric()),                       # The PEAK_FLOW columns can also only be numeric
    "PEAK_FLOW_3"   : (lambda peak_flow: str(peak_flow).isnumeric()),                       # The PEAK_FLOW columns can also only be numeric     
}

# Removing all rows which cannot be valid by using the valid_data dictionary
invalid_rows = set()   # Set is used to avoid duplicate values
for column in df.columns:
    index = 0       # Used to keep track of row idex as rows can be deleted, changing the indexes of all rows
    for field in df[column]:
        if not valid_data[column](field): 
            invalid_rows.add(index)
        index += 1

print(invalid_rows)
df.drop(invalid_rows, inplace=True)

# Removes any duplicate rows
df.drop_duplicates(inplace=True)

post_clean_count = len(df.index)

print(f"There were originally {pre_clean_count} rows.\n \
    After removing invalid rows, there are {post_clean_count} rows.\n \
    In total, {pre_clean_count - post_clean_count} rows have been dropped.")

{1536, 2050, 1539, 2051, 1545, 13, 2061, 1529, 532, 534, 2072, 1562, 1566, 2092, 48, 1584, 1590, 57, 2107, 2110, 2111, 576, 1603, 1605, 1094, 1607, 1609, 2126, 591, 1616, 1113, 1629, 2146, 1635, 612, 1637, 1642, 1650, 2170, 1150, 2174, 2178, 1677, 1166, 1167, 2193, 658, 1687, 2202, 1692, 2207, 1697, 1700, 1193, 171, 2220, 1197, 686, 2225, 1205, 2229, 2235, 2236, 1726, 1730, 708, 2254, 1744, 209, 730, 2271, 1251, 1255, 1263, 1265, 755, 245, 761, 1274, 768, 1796, 261, 1297, 1298, 278, 792, 1819, 285, 1821, 1313, 802, 1315, 294, 807, 296, 1319, 304, 1333, 828, 1353, 1354, 1869, 1358, 340, 1876, 1879, 1372, 1888, 1890, 356, 367, 368, 1909, 1922, 1928, 1935, 404, 1429, 406, 408, 1433, 411, 923, 413, 1441, 1957, 940, 1973, 438, 1467, 445, 1481, 973, 468, 470, 984, 1496, 2011, 991, 2018, 2027, 1517, 1519, 2039, 505}
There were originally 2284 rows.
     After removing invalid rows, there are 2086 rows.
     In total, 198 rows have been dropped.


Now it is time to clean the typos in the `AGE` column.
There are two main typos in the `AGE` column
The first typo is making the age negative, and the second typo is adding an extra 0.

In [86]:
pre_clean_age_minmax = min(df["AGE"]), max(df["AGE"])

for l in df["AGE"]:
    if l == 0:
        print(l)

df["AGE"] = df["AGE"].abs()     # Makes all the negative numbers positive
df["AGE"] = df["AGE"].map(lambda age: age // 10 if age > 150 else age)     # Removes the extra '0' from big numbers (e.g. 880 becomes 88)

print(df["AGE"])

post_clean_age_minmax = min(df["AGE"]), max(df["AGE"])

print(f"The minimum and maximum ages before cleaning were {pre_clean_age_minmax} respectively. \n \
    The minimum and maximum ages after cleaning were {post_clean_age_minmax} respectively.")

0       79
1       62
2       71
3       73
4       66
        ..
2275    77
2276    83
2277    63
2278    86
2279    58
Name: AGE, Length: 2086, dtype: int64
The minimum and maximum ages before cleaning were (-94, 880) respectively. 
     The minimum and maximum ages after cleaning were (28, 105) respectively.
